# Análisis

**Luis Fernando Ruiz**


---
# Parte 1: Value Iteration (Iteración de Valores)
---


## 1.1 Gridworld - Convergencia de Valores y Acciones

Se ejecutó el algoritmo de Value Iteration sobre el Gridworld de 10x10 con un factor de descuento \\\gamma = 0.9, observando los resultados en las iteraciones 5, 10, 15, 20, 30 y 50.


### Resultados Observados

**Iteraciones iniciales (5-10):**

- **Valores:** Los cambios en los valores son notorios (con un Delta máximo inicial alto), y la información de las recompensas (+1, -1) comienza a propagarse por el tablero.
- **Política (Convergencia temprana):** Sorprendentemente, **la política se estabiliza desde la iteración 5**. Las acciones sugeridas en los estados ya apuntan lógicamente hacia la recompensa positiva y evitan las negativas, y no sufren ninguna alteración en las iteraciones posteriores.

**Iteraciones intermedias (15-20):**

- **Valores:** Los valores continúan poblando todo el grid, reflejando una estimación más precisa del retorno esperado desde cada estado. Los valores negativos se expanden desde las zonas de penalización.
- **Política:** La política se mantiene absolutamente estable e idéntica a la observada desde la iteración 5.

**Iteraciones finales (30-50):**

- **Valores (Convergencia asintótica):** Los valores de los estados continúan ajustándose, pero los cambios son cada vez más pequeños (el Delta máximo se reduce a apenas 0.0031). La función de valor se acerca a su estado estacionario.
- **Política:** La política se mantiene **idéntica** a la observada desde las etapas iniciales (iteración 5). No hay cambios en las acciones recomendadas.

_Estos valores se pueden ver en el notebook de value_iteration_

### Análisis de Convergencia

**¿Cuándo convergen los valores?**
En esta implementación, los valores de los estados continúan refinándose incluso hasta la iteración 50. La convergencia estricta de la función de valor (donde los cambios son menores a un $\epsilon$ muy pequeño) requiere un número considerable de iteraciones.

**¿Cuándo convergen las acciones (política)?**
La política óptima converge muchísimo antes que los valores exactos, estabilizándose de forma casi inmediata desde la **iteración 5**.

**Conclusión:**
Esto demuestra empíricamente una propiedad fundamental de Value Iteration: no es necesario esperar a que los valores converjan perfectamente para extraer la política óptima. Una vez que el _orden relativo_ de los Q-valores para las acciones en un estado es correcto (ej. $Q(s, \text{'right'}) > Q(s, \text{'up'})$), la política derivada ya no cambiará, garantizando que el agente esté listo para tomar decisiones correctas desde etapas muy tempranas, incluso si los valores numéricos siguen actualizándose.


## 1.2 Bridge - Efecto del Factor de Descuento

Se evaluó el entorno Bridge (matriz 3x7 con puente en la fila central y penalizaciones -100 a los lados) utilizando factores de descuento \\gamma = 0.9$ y \\gamma = 0.1$ durante 10 iteraciones.


### Resultados Observados

**Con Factor de Descuento Alto (\\gamma = 0.9$)**

- **Valores:** Se observan valores positivos significativos a lo largo del camino del puente, incrementando a medida que se acercan a la meta (+100). Incluso en los estados iniciales, el valor refleja fuertemente la recompensa futura.
- **Política:** El agente elige decididamente el camino óptimo a través del puente, evitando las zonas de penalización.

**Con Factor de Descuento Bajo (\\gamma = 0.1$)**

- **Valores:** Los valores decaen drásticamente. Solo los estados inmediatamente adyacentes a la meta (+100) tienen valores positivos considerables. En estados más lejanos, el valor esperado de cruzar el puente es prácticamente nulo debido al fuerte descuento.
- **Política:** Aunque en nuestro entorno Bridge determinista el agente aún sigue el puente (porque es la única ruta hacia una recompensa sin riesgo inmediato), la "motivación" matemática es mucho menor.

### Análisis del Efecto del Factor de Descuento

El factor de descuento \\gamma$ define qué tan "miope" o "visionario" es el agente.

- **Agente Visionario (\\gamma = 0.9$):** Valora casi por igual las recompensas a largo plazo que las inmediatas. Esto fomenta estrategias que buscan recompensas grandes lejanas (como el +100 del puente), incluso si el camino es largo y no ofrece beneficios inmediatos. Los valores se propagan eficazmente por todo el estado del espacio.

- **Agente Miope (\\gamma = 0.1$):** Descuenta severamente el futuro. Una recompensa a varios pasos de distancia tiene un valor presente cercano a cero. Este agente prioriza fuertemente la gratificación instantánea o la evasión de riesgos inmediatos. En un entorno más complejo o estocástico, este bajo descuento podría llevar al agente a conformarse con recompensas subóptimas locales o a inmovilizarse por temor a penalizaciones lejanas.


---
# Parte 2: Policy Iteration (Iteración de Políticas)
---


## 2.1 Gridworld y Bridge - Convergencia de Políticas

Se ejecutó el algoritmo de Policy Iteration en ambos entornos para observar la rapidez con la que se alcanza la política óptima estable.


### Resultados Observados

**Gridworld 10x10 (\\gamma = 0.9$):**

- En la ejecución, la Iteración de Políticas reportó convergencia en apenas **2 iteraciones** externas (ciclos de evaluación-mejora).
- La política resultante es idéntica a la política final (y convergida) encontrada por Value Iteration después de 20+ iteraciones.

**Bridge Environment:**

- Tanto para \\gamma = 0.9$ como para \\gamma = 0.1$, la política convergió en **8 iteraciones** o menos.

### Análisis de Convergencia en Policy Iteration

**¿Por qué converge tan rápido Policy Iteration en términos de iteraciones externas?**
A diferencia de Value Iteration que actualiza gradualmente los valores basándose en un "horizonte" que se expande paso a paso, Policy Iteration realiza un esfuerzo computacional mayor en cada ciclo: evalúa _completamente_ el valor de la política actual hasta la convergencia antes de intentar mejorarla.

Al tener una evaluación precisa del rendimiento de la estrategia actual, el paso de _Policy Improvement_ puede hacer saltos mucho más directos y audaces hacia la política óptima en todo el espacio de estados simultáneamente. Esto resulta en un menor número de iteraciones principales necesarias para converger, aunque cada iteración individual sea más costosa.


---
# Conclusiones Finales
---

Basado en la implementación y experimentación con **Value Iteration** y **Policy Iteration**, concluimos lo siguiente:

1. **Mismo Destino, Diferente Camino:** Ambos algoritmos son capaces de encontrar exitosamente la política óptima para los MDPs planteados. Sin embargo, su dinámica de convergencia es distinta.
2. **Velocidad de Convergencia:**
   - **Value Iteration** converge iterativamente actualizando valores de estado localmente. Requiere más iteraciones en total, pero cada iteración es computacionalmente ligera. La política converge drásticamente antes de que los valores exactos lo hagan (en nuestro experimento, desde la iteración 5).
   - **Policy Iteration** converge en significativamente menos ciclos externos. Sin embargo, cada ciclo requiere evaluar completamente la política actual, lo cual es más costoso. Es ideal cuando se necesita llegar rápidamente a reglas de decisión estables.
3. **Impacto del Entorno:** El factor de descuento \\gamma$ demostró ser el hiperparámetro más influyente, alterando drásticamente la valoración del agente sobre el entorno (Bridge) y determinando si su estrategia es cortoplacista (miope) o enfocada en maximizar el retorno global a largo plazo (visionaria).
